In [7]:
import io
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# ----------------------------------------------------
# 1. VISÃO GERAL
# ----------------------------------------------------
# Agora, em vez de escolher "qual arquivo" vai fornecer o X e "qual arquivo" vai fornecer o Y,
# teremos uma lista única com todas as colunas de todos os arquivos.
#
#   - Dropdown de X: exibe todas as colunas (ex.: A (arquivo1.csv), B (arquivo1.csv), C (arquivo2.txt), etc.).
#   - Checkboxes de Y: exibe todas as colunas EXCETO a que foi selecionada para X.
#   - Cada coluna Y terá seu próprio painel de estilo (cor, tipo de linha, etc.).
#
# Um dicionário global "all_columns_map" relaciona:
#   "nomeColuna (nomeArquivo)" -> (nomeArquivo, nomeColuna)
#
# ----------------------------------------------------
# 2. PREPARAÇÃO DE WIDGETS E ESTRUTURAS GLOBAIS
# ----------------------------------------------------
default_layout = widgets.Layout(width='300px')
default_style = {'description_width': 'initial'}

def format_text(text, bold, italic):
    """
    Formata o texto com MathText, aplicando negrito e/ou itálico.
    """
    if bold and italic:
        return fr'$\textbf{{\textit{{{text}}}}}$'
    elif bold:
        return fr'$\textbf{{{text}}}$'
    elif italic:
        return fr'$\textit{{{text}}}$'
    else:
        return text

# Dicionário de DataFrames: cada arquivo carregado é armazenado aqui.
dfs = {}

# Mapa global de colunas: "nomeColuna (nomeArquivo)" -> (nomeArquivo, nomeColuna)
all_columns_map = {}

# ----------------------------------------------------
# 3. FUNÇÕES DE LEITURA DE ARQUIVOS
# ----------------------------------------------------
def simple_process_text_file(content, sep):
    """
    Lê um arquivo de texto (CSV ou TXT) ignorando linhas vazias.
    Retorna um DataFrame com todas as colunas lidas do arquivo, usando o cabeçalho existente.
    """
    try:
        text = content.decode('utf-8')
    except Exception:
        text = content.decode('latin1')
    lines = [line for line in text.splitlines() if line.strip() != ""]
    from io import StringIO
    df = pd.read_csv(StringIO("\n".join(lines)), header=0, sep=sep)
    return df

# ----------------------------------------------------
# 4. WIDGETS DE UPLOAD
# ----------------------------------------------------
upload_widget = widgets.FileUpload(
    accept='.csv, .xlsx, .txt',
    multiple=True,
    description='Carregar arquivos',
    layout=default_layout,
    style=default_style
)

output_area = widgets.Output()

# ----------------------------------------------------
# 5. SELEÇÃO DE COLUNAS (X e Y) - VISÃO GLOBAL
# ----------------------------------------------------
# Dropdown para Eixo X
dropdown_x = widgets.Dropdown(
    options=[],
    description='Eixo X:',
    disabled=True,
    layout=default_layout,
    style=default_style
)

# Checkboxes para seleção de colunas de Eixo Y
y_checkboxes = {}
y_checkboxes_box = widgets.VBox([])

# Dicionário para armazenar widgets de estilo para cada coluna Y selecionada
style_widgets_y = {}
dynamic_y_style_box = widgets.VBox([])

def create_line_style_widget(column_label):
    """
    Cria um painel de widgets para personalizar a linha associada a 'column_label'.
    """
    header = widgets.HTML(f"<b>Estilo para '{column_label}'</b>")

    label_text = widgets.Text(
        value=column_label,
        description='Legenda:',
        layout=default_layout,
        style=default_style
    )
    label_bold = widgets.Checkbox(
        value=False,
        description='Negrito',
        layout=widgets.Layout(width='100px'),
        style=default_style
    )
    label_italic = widgets.Checkbox(
        value=False,
        description='Itálico',
        layout=widgets.Layout(width='100px'),
        style=default_style
    )
    label_style_box = widgets.HBox([label_bold, label_italic])

    line_color = widgets.Dropdown(
        options=['azul', 'vermelho', 'preto', 'verde', 'laranja', 'roxo'],
        value='azul',
        description='Cor da linha:',
        layout=default_layout,
        style=default_style
    )
    line_style = widgets.Dropdown(
        options=['contínua', 'tracejada', 'pontilhada', 'traço e ponto'],
        value='contínua',
        description='Tipo da linha:',
        layout=default_layout,
        style=default_style
    )
    line_width = widgets.IntText(
        value=2,
        description='Espessura da linha:',
        layout=default_layout,
        style=default_style
    )
    marker_checkbox = widgets.Checkbox(
        value=False,
        description='Exibir marcadores',
        layout=default_layout,
        style=default_style
    )
    marker_type = widgets.Dropdown(
        options=['círculo', 'quadrado', 'triângulo', 'losango', 'estrela'],
        value='círculo',
        description='Tipo marcador:',
        layout=default_layout,
        style=default_style
    )
    marker_color = widgets.Dropdown(
        options=['azul', 'vermelho', 'preto', 'verde', 'laranja', 'roxo'],
        value='preto',
        description='Cor marcador:',
        layout=default_layout,
        style=default_style
    )
    marker_size = widgets.IntText(
        value=5,
        description='Tamanho marcador:',
        layout=default_layout,
        style=default_style
    )

    box = widgets.VBox([
        header,
        label_text,
        label_style_box,
        line_color,
        line_style,
        line_width,
        marker_checkbox,
        marker_type,
        marker_color,
        marker_size
    ])
    return {
        'panel': box,
        'label_text': label_text,
        'label_bold': label_bold,
        'label_italic': label_italic,
        'line_color': line_color,
        'line_style': line_style,
        'line_width': line_width,
        'marker_checkbox': marker_checkbox,
        'marker_type': marker_type,
        'marker_color': marker_color,
        'marker_size': marker_size
    }

def on_y_checkbox_change(change):
    """
    Atualiza o painel de estilo correspondente quando uma checkbox de Y é marcada/desmarcada.
    """
    column_label = change['owner'].description  # Ex.: "A (file1.csv)"
    if change['new']:
        if column_label not in style_widgets_y:
            style_widgets_y[column_label] = create_line_style_widget(column_label)
    else:
        if column_label in style_widgets_y:
            del style_widgets_y[column_label]
    dynamic_y_style_box.children = [style_widgets_y[key]['panel'] for key in style_widgets_y]

def update_y_checkboxes():
    """
    Reconstrói as checkboxes de Y com base em all_columns_map,
    excluindo a coluna atualmente selecionada para X.
    """
    chosen_x = dropdown_x.value
    new_boxes = []
    y_checkboxes.clear()
    style_widgets_y.clear()
    for col_label in all_columns_map.keys():
        if col_label == chosen_x:
            continue  # Não exibe a coluna escolhida para X
        cb = widgets.Checkbox(
            value=False,
            description=col_label,
            layout=default_layout,
            style=default_style
        )
        cb.observe(on_y_checkbox_change, names='value')
        y_checkboxes[col_label] = cb
        new_boxes.append(cb)
    y_checkboxes_box.children = new_boxes
    dynamic_y_style_box.children = []

def on_x_change(change):
    """
    Quando o eixo X é alterado, atualiza as checkboxes de Y.
    """
    if change['name'] == 'value':
        update_y_checkboxes()

dropdown_x.observe(on_x_change, names='value')

# ----------------------------------------------------
# 6. BOTÃO PARA LIMPAR DADOS CARREGADOS
# ----------------------------------------------------
clear_button = widgets.Button(
    description='Limpar Dados',
    button_style='warning',
    layout=widgets.Layout(width='150px')
)

def clear_data(b):
    global dfs, all_columns_map
    dfs.clear()
    all_columns_map.clear()
    upload_widget.value.clear()
    upload_widget._counter = 0

    dropdown_x.options = []
    dropdown_x.disabled = True
    y_checkboxes_box.children = []
    dynamic_y_style_box.children = []
    style_widgets_y.clear()

    with output_area:
        clear_output()
        print("Dados limpos.")

clear_button.on_click(clear_data)

# ----------------------------------------------------
# 7. WIDGETS DE TRATAMENTO DE DADOS
# ----------------------------------------------------
missing_checkbox = widgets.Checkbox(
    value=False,
    description='Remover valores faltantes',
    layout=default_layout,
    style=default_style
)
missing_method = widgets.Dropdown(
    options=['Remover linha', 'Substituir pela média', 'Substituir pelo valor anterior'],
    description='Tratamento faltantes:',
    layout=default_layout,
    style=default_style
)
normalize_checkbox = widgets.Checkbox(
    value=False,
    description='Normalizar dados',
    layout=default_layout,
    style=default_style
)
normalize_method = widgets.Dropdown(
    options=['Min-Max [0,1]', 'Z-score', 'Nenhuma'],
    description='Tipo normalização:',
    layout=default_layout,
    style=default_style
)
normalization_target = widgets.Dropdown(
    options=['X', 'Y', 'Ambos'],
    description='Aplicar normalização em:',
    layout=default_layout,
    style=default_style
)
transform_dropdown = widgets.Dropdown(
    options=['Nenhuma', 'Log', 'Raiz Quadrada'],
    description='Transformação:',
    layout=default_layout,
    style=default_style
)
transformation_target = widgets.Dropdown(
    options=['X', 'Y', 'Ambos'],
    description='Aplicar transformação em:',
    layout=default_layout,
    style=default_style
)
filter_x_text = widgets.Text(
    value='Nenhum',
    description='Filtrar eixo X (mín,máx):',
    placeholder='ex: 0,100 ou Nenhum',
    layout=default_layout,
    style=default_style
)
filter_y_text = widgets.Text(
    value='Nenhum',
    description='Filtrar eixo Y (mín,máx):',
    placeholder='ex: 0,100 ou Nenhum',
    layout=default_layout,
    style=default_style
)
moving_avg_checkbox = widgets.Checkbox(
    value=False,
    description='Aplicar média móvel',
    layout=default_layout,
    style=default_style
)
moving_avg_window = widgets.IntText(
    value=5,
    description='Tamanho da janela:',
    disabled=True,
    layout=default_layout,
    style=default_style
)

def on_moving_avg_change(change):
    moving_avg_window.disabled = not change['new']
moving_avg_checkbox.observe(on_moving_avg_change, names='value')

# ----------------------------------------------------
# 8. CONFIGURAÇÃO VISUAL GLOBAL
# ----------------------------------------------------
title_text = widgets.Text(
    value='Meu Gráfico',
    description='Título:',
    layout=default_layout,
    style=default_style
)
title_bold_checkbox = widgets.Checkbox(
    value=False,
    description='Negrito',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
title_italic_checkbox = widgets.Checkbox(
    value=False,
    description='Itálico',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
title_style_box = widgets.HBox([title_bold_checkbox, title_italic_checkbox])

xlabel_text = widgets.Text(
    value='Eixo X',
    description='Rótulo X:',
    layout=default_layout,
    style=default_style
)
xlabel_bold_checkbox = widgets.Checkbox(
    value=False,
    description='Negrito',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
xlabel_italic_checkbox = widgets.Checkbox(
    value=False,
    description='Itálico',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
xlabel_style_box = widgets.HBox([xlabel_bold_checkbox, xlabel_italic_checkbox])

ylabel_text = widgets.Text(
    value='Eixo Y',
    description='Rótulo Y:',
    layout=default_layout,
    style=default_style
)
ylabel_bold_checkbox = widgets.Checkbox(
    value=False,
    description='Negrito',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
ylabel_italic_checkbox = widgets.Checkbox(
    value=False,
    description='Itálico',
    layout=widgets.Layout(width='100px'),
    style=default_style
)
ylabel_style_box = widgets.HBox([ylabel_bold_checkbox, ylabel_italic_checkbox])

x_limits_text = widgets.Text(
    value='Nenhum',
    description='Limites Eixo X (min,máx):',
    placeholder='ex: 0,100 ou Nenhum',
    layout=default_layout,
    style=default_style
)
y_limits_text = widgets.Text(
    value='Nenhum',
    description='Limites Eixo Y (min,máx):',
    placeholder='ex: 0,1 ou Nenhum',
    layout=default_layout,
    style=default_style
)
grid_checkbox = widgets.Checkbox(
    value=False,
    description='Exibir grade',
    layout=default_layout,
    style=default_style
)
grid_axes = widgets.Dropdown(
    options=['X', 'Y', 'Ambos'],
    value='Ambos',
    description='Eixos da grade:',
    layout=default_layout,
    style=default_style
)
legend_checkbox = widgets.Checkbox(
    value=False,
    description='Exibir legenda',
    layout=default_layout,
    style=default_style
)
font_size = widgets.IntText(
    value=12,
    description='Tamanho da fonte:',
    layout=default_layout,
    style=default_style
)

# ----------------------------------------------------
# 9. FUNÇÃO PARA PROCESSAR SERIES (TRATAMENTO, NORMALIZAÇÃO, ETC.)
# ----------------------------------------------------
def process_series(serie, is_x=False):
    # 1) Faltantes
    if missing_checkbox.value:
        if missing_method.value == 'Remover linha':
            serie = serie.dropna()
        elif missing_method.value == 'Substituir pela média':
            serie = serie.fillna(serie.mean())
        elif missing_method.value == 'Substituir pelo valor anterior':
            serie = serie.fillna(method='ffill')
    # 2) Normalização
    if normalize_checkbox.value and normalize_method.value != 'Nenhuma':
        if (is_x and normalization_target.value in ['X', 'Ambos']) or (not is_x and normalization_target.value in ['Y', 'Ambos']):
            if normalize_method.value == 'Min-Max [0,1]':
                denom = serie.max() - serie.min()
                if denom != 0:
                    serie = (serie - serie.min()) / denom
            elif normalize_method.value == 'Z-score':
                stdev = serie.std()
                if stdev != 0:
                    serie = (serie - serie.mean()) / stdev
    # 3) Transformação
    if transform_dropdown.value != 'Nenhuma':
        if (is_x and transformation_target.value in ['X', 'Ambos']) or (not is_x and transformation_target.value in ['Y', 'Ambos']):
            if transform_dropdown.value == 'Log':
                serie = serie.apply(lambda x: np.log(x) if x > 0 else np.nan)
            elif transform_dropdown.value == 'Raiz Quadrada':
                serie = serie.apply(lambda x: np.sqrt(x) if x >= 0 else np.nan)
    # 4) Filtros
    if is_x:
        if filter_x_text.value.strip().lower() != 'nenhum':
            try:
                min_val, max_val = map(float, filter_x_text.value.split(','))
                serie = serie[(serie >= min_val) & (serie <= max_val)]
            except:
                pass
    else:
        if filter_y_text.value.strip().lower() != 'nenhum':
            try:
                min_val, max_val = map(float, filter_y_text.value.split(','))
                serie = serie[(serie >= min_val) & (serie <= max_val)]
            except:
                pass
    # 5) Média móvel (apenas para Y)
    if not is_x and moving_avg_checkbox.value:
        serie = serie.rolling(window=moving_avg_window.value, min_periods=1).mean()
    return serie

# ----------------------------------------------------
# 10. FUNÇÃO PRINCIPAL DE PLOTAGEM
# ----------------------------------------------------
ls_map = {
    'contínua': '-',
    'tracejada': '--',
    'pontilhada': ':',
    'traço e ponto': '-.'
}
color_map = {
    'azul': 'blue',
    'vermelho': 'red',
    'preto': 'black',
    'verde': 'green',
    'laranja': 'orange',
    'roxo': 'purple'
}
marker_map = {
    'círculo': 'o',
    'quadrado': 's',
    'triângulo': '^',
    'losango': 'D',
    'estrela': '*'
}

def plot_graph(b):
    with output_area:
        clear_output()
        if not all_columns_map:
            print("Nenhuma coluna disponível. Por favor, carregue arquivos.")
            return
        chosen_x = dropdown_x.value
        if not chosen_x:
            print("Por favor, selecione uma coluna para o eixo X.")
            return
        file_x, col_x = all_columns_map[chosen_x]
        s_x = dfs[file_x][col_x].copy()
        s_x = process_series(s_x, is_x=True)
        plt.figure(figsize=(8, 5))
        any_plotted = False
        for col_label, cb in y_checkboxes.items():
            if cb.value:
                file_y, col_y = all_columns_map[col_label]
                s_y = dfs[file_y][col_y].copy()
                s_y = process_series(s_y, is_x=False)
                common_idx = s_x.index.intersection(s_y.index)
                s_x_plot = s_x.loc[common_idx]
                s_y_plot = s_y.loc[common_idx]
                style = style_widgets_y[col_label] if col_label in style_widgets_y else {}
                label_text_formatted = format_text(
                    style.get('label_text', widgets.Text(value=col_label)).value,
                    style.get('label_bold', widgets.Checkbox(value=False)).value,
                    style.get('label_italic', widgets.Checkbox(value=False)).value
                )
                line_col = color_map.get(style.get('line_color', widgets.Dropdown(value='azul')).value, 'blue')
                line_ls = ls_map.get(style.get('line_style', widgets.Dropdown(value='contínua')).value, '-')
                lw = style.get('line_width', widgets.IntText(value=2)).value
                if style.get('marker_checkbox', widgets.Checkbox(value=False)).value:
                    mk = marker_map.get(style.get('marker_type', widgets.Dropdown(value='círculo')).value, 'o')
                    ms = style.get('marker_size', widgets.IntText(value=5)).value
                    plt.plot(
                        s_x_plot, s_y_plot,
                        linestyle=line_ls,
                        color=line_col,
                        linewidth=lw,
                        marker=mk,
                        markersize=ms,
                        label=label_text_formatted
                    )
                else:
                    plt.plot(
                        s_x_plot, s_y_plot,
                        linestyle=line_ls,
                        color=line_col,
                        linewidth=lw,
                        label=label_text_formatted
                    )
                any_plotted = True
        if not any_plotted:
            print("Nenhuma coluna foi selecionada para Y.")
            return
        if x_limits_text.value.strip().lower() != 'nenhum':
            try:
                xmin, xmax = map(float, x_limits_text.value.split(','))
                plt.xlim(xmin, xmax)
            except:
                pass
        if y_limits_text.value.strip().lower() != 'nenhum':
            try:
                ymin, ymax = map(float, y_limits_text.value.split(','))
                plt.ylim(ymin, ymax)
            except:
                pass
        title_str = format_text(title_text.value, title_bold_checkbox.value, title_italic_checkbox.value)
        plt.title(title_str, fontsize=font_size.value + 2)
        xlabel_str = format_text(xlabel_text.value, xlabel_bold_checkbox.value, xlabel_italic_checkbox.value)
        plt.xlabel(xlabel_str, fontsize=font_size.value)
        ylabel_str = format_text(ylabel_text.value, ylabel_bold_checkbox.value, ylabel_italic_checkbox.value)
        plt.ylabel(ylabel_str, fontsize=font_size.value)
        if grid_checkbox.value:
            if grid_axes.value == 'X':
                plt.grid(axis='x')
            elif grid_axes.value == 'Y':
                plt.grid(axis='y')
            else:
                plt.grid(True)
        if legend_checkbox.value:
            plt.legend(fontsize=font_size.value)
        plt.show()
        if save_checkbox.value:
            fmt = export_format.value.lower()
            filename = file_name_text.value + '.' + fmt
            plt.savefig(filename)
            print(f"Gráfico salvo como {filename}.")

plot_button = widgets.Button(
    description='Plotar Gráfico',
    button_style='success',
    layout=widgets.Layout(width='150px')
)
plot_button.on_click(plot_graph)

# ----------------------------------------------------
# 11. FUNÇÃO PARA ATUALIZAR O MAPA DE COLUNAS
# ----------------------------------------------------
def build_all_columns_map():
    """
    Constrói o dicionário all_columns_map com rótulos do tipo
    "nomeColuna (nomeArquivo)" -> (nomeArquivo, nomeColuna)
    """
    all_columns_map.clear()
    for file_name, df in dfs.items():
        for col in df.columns:
            label = f"{col} ({file_name})"
            all_columns_map[label] = (file_name, col)

def update_x_dropdown():
    """
    Atualiza o dropdown do eixo X com todas as colunas disponíveis.
    """
    if all_columns_map:
        dropdown_x.options = list(all_columns_map.keys())
        dropdown_x.disabled = False
    else:
        dropdown_x.options = []
        dropdown_x.disabled = True

# ----------------------------------------------------
# 12. FUNÇÃO DE UPLOAD
# ----------------------------------------------------
def on_upload_change(change):
    global dfs
    new_files = change['new']
    if new_files:
        # Itera sobre os novos arquivos carregados
        for file_info in new_files.values():
            name = file_info['metadata']['name']
            # Garante nomes únicos para evitar sobrescritas
            original_name = name
            counter = 1
            while name in dfs:
                name = f"{original_name} ({counter})"
                counter += 1
            content = file_info['content']
            try:
                if name.lower().endswith('.csv'):
                    df = simple_process_text_file(content, sep=',')
                elif name.lower().endswith('.txt'):
                    df = simple_process_text_file(content, sep='	')
                elif name.lower().endswith('.xlsx'):
                    df = pd.read_excel(io.BytesIO(content))
                else:
                    with output_area:
                        clear_output()
                        print(f"Formato do arquivo {name} não suportado.")
                    continue
                # Acumula o DataFrame no dicionário global
                dfs[name] = df
                with output_area:
                    print(f"Arquivo '{name}' carregado com sucesso!")
            except Exception as e:
                with output_area:
                    print(f"Erro ao ler o arquivo {name}: {e}")
        # Constrói o mapa de colunas com todos os arquivos carregados
        build_all_columns_map()
        update_x_dropdown()
        update_y_checkboxes()
        # Limpa o valor do widget para permitir uploads futuros
        upload_widget.value.clear()

upload_widget.observe(on_upload_change, names='value')

# ----------------------------------------------------
# 13. WIDGETS DE EXPORTAÇÃO
# ----------------------------------------------------
save_checkbox = widgets.Checkbox(
    value=False,
    description='Salvar gráfico',
    layout=default_layout,
    style=default_style
)
export_format = widgets.Dropdown(
    options=['PNG', 'PDF', 'SVG', 'JPG'],
    value='PNG',
    description='Formato:',
    layout=default_layout,
    style=default_style
)
file_name_text = widgets.Text(
    value='grafico',
    description='Nome do arquivo:',
    layout=default_layout,
    style=default_style
)

# ----------------------------------------------------
# 14. LAYOUT FINAL
# ----------------------------------------------------
upload_box = widgets.VBox([
    widgets.HTML("<h3>1. Carregamento de Dados</h3>"),
    upload_widget,
    clear_button
])
x_box = widgets.VBox([
    widgets.HTML("<h3>2. Selecione Coluna para Eixo X</h3>"),
    dropdown_x
])
y_selection_box = widgets.VBox([
    widgets.HTML("<h3>3. Selecione Colunas para Eixo Y</h3>"),
    y_checkboxes_box,
    widgets.HTML("<b>Personalização de cada Y:</b>"),
    dynamic_y_style_box
])
treatment_box = widgets.VBox([
    widgets.HTML("<h3>4. Tratamento de Dados</h3>"),
    missing_checkbox,
    missing_method,
    normalize_checkbox,
    normalize_method,
    normalization_target,
    transform_dropdown,
    transformation_target,
    filter_x_text,
    filter_y_text,
    moving_avg_checkbox,
    moving_avg_window
])
visual_box = widgets.VBox([
    widgets.HTML("<h3>5. Configuração Visual Global</h3>"),
    widgets.HTML("<b>Título do Gráfico</b>"),
    title_text,
    title_style_box,
    widgets.HTML("<b>Rótulo Eixo X</b>"),
    xlabel_text,
    xlabel_style_box,
    widgets.HTML("<b>Rótulo Eixo Y</b>"),
    ylabel_text,
    ylabel_style_box,
    x_limits_text,
    y_limits_text,
    grid_checkbox,
    grid_axes,
    legend_checkbox,
    font_size
])
export_box = widgets.VBox([
    widgets.HTML("<h3>6. Exportação</h3>"),
    save_checkbox,
    export_format,
    file_name_text
])
ui = widgets.VBox([
    upload_box,
    x_box,
    y_selection_box,
    treatment_box,
    visual_box,
    plot_button,
    export_box,
    output_area
])

display(ui)
